<a href="https://colab.research.google.com/github/BriefCud/PRA3024-Big-Data-in-Phys/blob/master/Gravitational_Waves_Week_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code For the Exercises

In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Mar  2 12:06:33 2020

@author: Alexander
"""

#############################################################
#################### Importing libraries ####################
#############################################################
import numpy as np
import math as m
import time

#############################################################
################ Declaring Global Variables #################
#############################################################

#Speed of light in a vacuum
c = 299792458 #m•s^-1

#Gravitational constant
G = 6.67430 * 10**(-11) #m^3•kg^-1•s^-2

#Upper and Lower limits for the fisher integration
upper =  1
lower = 1
fs = 1

#The fourier transform variable
F = 420

SNR = 10

#other variable
A = 1

#The initial parameters
t = 0
phi = 0
eta = 0
mass = 0

initParams = [t,phi,eta,mass]

#The alpha coefficients
alpha = []
alpha.append(1)
alpha.append(0)
alpha.append((-16)*m.pi)
alpha.append(10 * (3058673/1016064))

#############################################################
#################### Declaring Functions ####################
#############################################################
'''
This function calculates A using the trapezoidla rule where,
∫ƒ(x) ≈ h * (0.5 * (ƒ(x 0)+f(x n))) + ƒ(x 1) + ƒ(x 2) + ... + ƒ(x n-1))) 
There is a small error in this calculation due to this method but I will negate it
'''
def GetA():
    #This is the equation i need to intergrate 'y = (f**(-7/6))/AdvLigoS(f)'
    #The variable 'n' is the number of divisions we want to have
    #The more divisions the lower the error in the result hence n should be large
    integral = 0
    n=1000
    h = float((upper-lower) / n)
    integral = 0.5 * ((lower**(-7/6))/AdvLigoS(lower) + (upper**(-7/6))/AdvLigoS(upper))
    for i in range(int(lower), int(upper)):
      integral += ((lower + i*h)**(-7/6))/AdvLigoS(lower + i*h)
    return np.sqrt(SNR / (integral * h * 4))
'''
'''
def H(params):
    zum = 0
    t,phi,eta,mass = params

    #making this value dimensionless by multiplying it by G * c^-3
    ni = (m.pi * mass * F * G * c**(-3))**(1/3) 
    
    #The sum of the products of the indexes in the alpha array and ni^(current index)
    for i in range(len(alpha)):
        zum += alpha[i] * ni**(i)
    
    psi = (2 * m.pi * F * t) - phi - (m.pi/4) + (3/(128 * eta * (ni**(5))) * zum)
    
    #function of the fourier variable
    hf = A * (F**(-7/6)) * (m.e**(psi * 1j))
    
    # print("-----------------TESTING PURPOSES-----------------")
    # print("-----------------def H(params)-----------------")
    # print("A   : ",A)
    # print("zum : ", zum)
    # print("psi : ", psi)
    # print("hf  : ",hf)
    # print("----------------------------------")
    
    return hf


def AdvLigoS(f):
    '''Function that computes the noise curve for the advanced LIGO'''
    f0 = 215
    global fs
    fs = 40
    s0 = 10**(-49) #Hz^-1
    x = f / f0

    #Setting the lower limit needed for the fisher matrix integration
    global lower
    lower = fs
    
    if (f >= fs): 
        #equation for the advanced LIGO noise curve found at 3.7
        shf = s0 * ((x**(-4.14)) - (5 * (x**(-2))) + ((111 * (1 - (x**(2)) + (x**(4)/2))) / ( 1 + ((x**(2))/2)))) 
        # print("-----------------TESTING PURPOSES-----------------")
        # print("------------PDS AdvLigoS(f) function--------------")
        # print("shf : ", shf)
        # print("----------------------------------")
        return shf
    else:
        return m.inf

'''
This function finds the partial derivative of h(f)
Using the equation ƒ´(x) = lim h->0 (ƒ(x + h) + ƒ(x))/h
where h is the difference between the two 'x' points that are taken
'''
def Hprime(n):    
    temp = initParams.copy()
    h = initParams[n]/100
    temp[n] += h
    derivative = (H(temp) - H(initParams)) / h

    # print("-----------------TESTING PURPOSES-----------------")
    # print("----------------Hprime(n) function----------------")
    # print("initParams")
    # print("t             : %s \nphi           : %s \neta           : %s \nmass          : %s" % (initParams[0], initParams[1], initParams[2], initParams[3]))
    # print("temp params")
    # print("t             : %s \nphi           : %s \neta           : %s \nmass          : %s" % (temp[0], temp[1], temp[2], temp[3]))
    # print("derivative    : ", derivative)
    # print("H(temp)       : ", (derivative * h + H(initParams)))
    # print("H(initParams):, ", (H(temp) - (derivative * h)))
    # print("----------------------------------")
    return derivative

'''
This function will create the fisher matrix and
use the trapezoidal rule to integrate, which is explained in the 'GetA()' function.
∫ƒ(x) ≈ h * (0.5 * (ƒ(x 0)+f(x n))) + ƒ(x 1) + ƒ(x 2) + ... + ƒ(x n-1))) 
''' 
def CreateFisherMatrix():
    matrix = []
    n = 1000
    h = float((upper-lower) / n)
    for i in range(len(initParams)):
        matrix.append([])
        for j in range(len(initParams)):
            integral = 0
            integral = np.real(0.5 * (Hprime(i)*Hprime(j) / AdvLigoS(lower)) + (Hprime(i)*Hprime(j) / AdvLigoS(upper)))
            for k in range(int(lower), int(upper)):
                integral += np.real(Hprime(i)*Hprime(j) / AdvLigoS((lower + k * h)))
            matrix[-1].append(4*integral*h)
    matrix = np.array(matrix)        
    return matrix 
    
    
'''
Main function
'''
def Main():
    
    '''
    As black holes vary in mass so much there is no average mass of a black hole
    therefore I have chosen the extragalactic black hole GW150914 which has a 
    solar mass of 36, where 1 solar mass = 2 * 10^30 Kg
    '''
    m1 = 36 * (2 * (10**30))
    m2 = 36 * (2 * (10**30))
    
#    m1 = input("Mass of first black hole in solar masses: ")
#    m2 = input("Mass of second black hole in solar masses: ")
#    m1, m2 = m1 * (2 * 10**30), m2 * (2 * 10**30)
    
    #The product of the masses
    mass = m1 + m2
    
    #Mass ratio
    eta = (m1 * m2) / (mass**(2))
  
    #Chirp mass
    chirpMass = eta**(3/5) * mass
    
    #Setting the initial parameters
    global initParams
    initParams[0] = 1    #t
    initParams[1] = 1    #phi
    initParams[2] = eta  #eta
    initParams[3] = mass #mass
    
    #Setting the upper limit needed for the fisher matrix integration
    #Multiplying the upper limit by G * c^-3 for the units to become Hz(s^-1)
    global upper
    upper = ( 6**(3/2) * m.pi * mass * G * c**(-3) )**-1

    #Getting A
    global A
    A = GetA()
    
    print("#####################################################################")
    print("#####################################################################")
    print("# EXERCISE 1 EXERCISE 1 EXERCISE 1 EXERCISE 1 EXERCISE 1 EXERCISE 1 #")
    print("#####################################################################")
    print("#####################################################################")
    print("")
    print("A  :  ", A)
    print("")
    print("")

    fisherMatrix = CreateFisherMatrix()

    print("Creating fisher information matrix...")
    time.sleep(1.5)
    print("")
    print("Fisher Information Matrix")
    print(fisherMatrix)
    print("")
    print("")

    inverse = np.linalg.inv(fisherMatrix)

    print("Inversing the matrix...")
    time.sleep(1.5)
    print("")
    print("Inverse of the matrix")
    print(inverse)
    print("")
    print("")

    diagonal = np.sqrt(np.abs(np.diagonal(inverse)))

    print("Taking out the n diagonal elements and taking the square root of each...")
    time.sleep(1.5)
    print("")
    print("Final Output for 2PN")
    print(diagonal)
    print("")

Main()


#####################################################################
#####################################################################
# EXERCISE 1 EXERCISE 1 EXERCISE 1 EXERCISE 1 EXERCISE 1 EXERCISE 1 #
#####################################################################
#####################################################################

A  :   1.103824888312316e-22


Creating fisher information matrix...

Fisher Information Matrix
[[-2.75888467e+00  2.19411023e-02 -4.41678616e-01 -9.72841244e-34]
 [ 2.19411023e-02 -1.01258179e-04  2.11685073e-03  4.70019579e-36]
 [-4.41678616e-01  2.11685073e-03 -4.41087655e-02 -9.78709453e-35]
 [-9.72841244e-34  4.70019579e-36 -9.78709453e-35 -2.17130436e-67]]


Inversing the matrix...

Inverse of the matrix
[[-2.49508885e+13  4.78040172e+16  7.09356786e+14  8.26858169e+47]
 [ 4.59065927e+16 -8.76217474e+19 -1.21288479e+18 -1.55571472e+51]
 [ 1.81870421e+14 -2.56204779e+17  2.04736845e+16 -1.55893508e+49]
 [ 1.02354788e+48 -1.99543690e+51 